In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
import json
import requests
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
def gen_url(n):
    base_url = "https://api.github.com"
    owner = "ironhack-datalabs"
    repo = "datamad0820"
    query = "?per_page=100&page={n}&state=closed"
    endpoint = f"/repos/{owner}/{repo}/pulls"
    return f"{base_url}{endpoint}{query}"

load_dotenv()
authkey = os.getenv("AUTHKEY")
headers = {"Authorization": f"Bearer {authkey}"}

In [13]:
gen_url_res = []

for page in range(1,6):
    gen_url_res += requests.get(gen_url(page),headers=headers).json()
 


In [14]:
print(len(gen_url_res))

500


In [39]:
def getPullHtml(number):
    res = requests.get(f'https://github.com/ironhack-datalabs/datamad0820/pull/{number}')
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def pullMemeLst(soup):
    images_lst = []
    images = soup.find_all("img")
    for x in images:
        if "user-images" in x['src']:
            images_lst.append(x['src'])
    return images_lst
    
def pullMentions(soup):
    user_mentions = set()
    names = soup.find_all("a")
    for x in names:
        if x.has_attr("class") and "user-mention" in x['class']:
            user_mentions.add(x.getText().split('@')[1])
    return user_mentions

#Tiene en cuenta tanto los usuarios mencionados con la @ como los de los join, porque los TA mencionan con la @ al 
#contestar 

In [43]:
#Ejemplo
#print(pullMentions(getPullHtml(385)))

#output: {'jorge-alamillos'}




{'jorge-alamillos'}


In [41]:
lst = {}
for pull in gen_url_res:
    number = pull["number"]
    creator = pull["user"]["login"]
    soup = getPullHtml(number)
    meme_lst= pullMemeLst(soup)
    
    if creator in lst:
        lst[creator] = lst[creator]+ meme_lst
    else:
        lst.update({creator : meme_lst})
    mentions = pullMentions(soup)
    for mention in mentions:
        if mention in lst:
            lst[mention] = lst[mention]+ meme_lst
        else:
            lst.update({mention : meme_lst})
            
            
    

In [44]:
#lst

{'AnaMA96'}
